In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn import metrics
import sklearn.ensemble as en
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,5)
a=['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack','last_flag']
df=pd.read_csv("D:\python\Train.txt",names=a)

In [4]:
df=df.replace(['neptune','pod','back','land','smurf','teardrop','apache2','processtable','worm','udpstorm'],'dos')
df=df.replace(['satan','ipsweep','portsweep','nmap','mscan','saint'],'probe')
df=df.replace(['named','sendmail','guess_passwd','imap','warezmaster','phf','spy','httptunnel','ftp_write','warezclient','xlock','xsnoop','snmpguess','snmpgetattack',],'r2l')
df=df.replace(['loadmodule','perl','rootkit','buffer_overflow','multihop','ps','sqlattack','mailbomb','xterm'],'u2r')

In [5]:
#encoding the categorical variables
categorical_col = []
for column in df.columns:
    if df[column].dtype == object and len(df[column].unique()) <= 500:
        categorical_col.append(column)
for i in categorical_col:
    df[i] = df[i].astype("category").cat.codes

In [6]:
x=df[df.columns.difference(['attack'])]
y=df['attack']
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.3,random_state=42)

### DECISION TREE

In [7]:
bg=BaggingClassifier(oob_score=True,n_estimators=3,random_state=234,n_jobs=-1,verbose=True)
bg.fit(train_x,train_y)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    5.8s finished
E:\programs\lib\site-packages\sklearn\ensemble\_bagging.py:633: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
E:\programs\lib\site-packages\sklearn\ensemble\_bagging.py:638: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])


BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=3, n_jobs=-1,
                  oob_score=True, random_state=234, verbose=True,
                  warm_start=False)

In [8]:
metrics.accuracy_score(test_y,bg.predict(test_x))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.2s finished


0.9979096104995766

In [9]:
from sklearn.metrics import confusion_matrix,classification_report

In [10]:
print(classification_report(test_y,bg.predict(test_x)))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13825
           1       1.00      1.00      1.00     20083
           2       1.00      0.99      0.99      3540
           3       0.98      0.96      0.97       326
           4       0.58      0.61      0.59        18

    accuracy                           1.00     37792
   macro avg       0.91      0.91      0.91     37792
weighted avg       1.00      1.00      1.00     37792



[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.2s finished


In [11]:
bg2=BaggingClassifier(oob_score=True,n_estimators=3,random_state=1234,n_jobs=-1,verbose=True)
bg2.fit(train_x,train_y)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    2.2s finished
E:\programs\lib\site-packages\sklearn\ensemble\_bagging.py:633: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
E:\programs\lib\site-packages\sklearn\ensemble\_bagging.py:638: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])


BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=3, n_jobs=-1,
                  oob_score=True, random_state=1234, verbose=True,
                  warm_start=False)

In [12]:
metrics.accuracy_score(test_y,bg2.predict(test_x))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished


0.9982271380186283

## Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
rf=RandomForestClassifier(oob_score=True,n_estimators=3,random_state=1234,n_jobs=-1,verbose=True,max_features=7)
rf.fit(train_x,train_y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.4s finished
E:\programs\lib\site-packages\sklearn\ensemble\_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
E:\programs\lib\site-packages\sklearn\ensemble\_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=7,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3, n_jobs=-1,
                       oob_score=True, random_state=1234, verbose=True,
                       warm_start=False)

In [15]:
metrics.accuracy_score(train_y,rf.predict(train_x))

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


0.9997391728376861

In [16]:
metrics.accuracy_score(test_y,rf.predict(test_x))

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


0.9987298899237934

## binomial classification

In [17]:
a=['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack','last_flag']
af=pd.read_csv("D:\python\Train.txt",names=a)

In [18]:
af['attack'] = np.where(af['attack']!='normal', 'other', af['attack'])


In [19]:
categorical_col = []
for column in af.columns:
    if af[column].dtype == object and len(af[column].unique()) <= 500:
        categorical_col.append(column)
for i in categorical_col:
    af[i] = af[i].astype("category").cat.codes

In [20]:
x=af[af.columns.difference(['attack'])]
y=af['attack']
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.3,random_state=42)

In [21]:
bg.fit(train_x,train_y)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    2.0s finished
E:\programs\lib\site-packages\sklearn\ensemble\_bagging.py:633: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
E:\programs\lib\site-packages\sklearn\ensemble\_bagging.py:638: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])


BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=3, n_jobs=-1,
                  oob_score=True, random_state=234, verbose=True,
                  warm_start=False)

In [22]:
bg2.fit(train_x,train_y)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.9s finished
E:\programs\lib\site-packages\sklearn\ensemble\_bagging.py:633: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
E:\programs\lib\site-packages\sklearn\ensemble\_bagging.py:638: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])


BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=3, n_jobs=-1,
                  oob_score=True, random_state=1234, verbose=True,
                  warm_start=False)

In [23]:
metrics.roc_auc_score(train_y,bg2.predict(train_x))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished


0.9997703767391445

In [24]:
metrics.roc_auc_score(train_y,bg.predict(train_x))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished


0.9998314700653095

In [25]:
metrics.accuracy_score(train_y,bg.predict(train_x))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished


0.9998298953289257

In [26]:
metrics.roc_auc_score(test_y,bg.predict(test_x))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished


0.9990652027470153

In [27]:
scoring=metrics.accuracy_score

skfold=StratifiedKFold(n_splits=10,random_state=123)
results = cross_val_score(bg, x,y, cv=skfold)

results.mean()

### testing

In [42]:
tf=pd.read_csv("D:\python\Test.txt",names=a)
tf=tf.replace(['neptune','pod','back','land','smurf','teardrop','apache2','processtable','worm','udpstorm'],'dos')
tf=tf.replace(['satan','ipsweep','portsweep','nmap','mscan','saint'],'probe')
tf=tf.replace(['named','sendmail','guess_passwd','imap','warezmaster','phf','spy','httptunnel','ftp_write','warezclient','xlock','xsnoop','snmpguess','snmpgetattack',],'r2l')
tf=tf.replace(['loadmodule','perl','rootkit','buffer_overflow','multihop','ps','sqlattack','mailbomb','xterm'],'u2r')

In [45]:
tf['attack'] = tf.attack.astype("category").cat.codes
tf['protocol_type'] = tf.attack.astype("category").cat.codes
tf['flag'] = tf.attack.astype("category").cat.codes
tf['service']=tf.attack.astype("category").cat.codes

In [46]:
x=tf[tf.columns.difference(['attack'])]
y=tf['attack']

In [32]:
metrics.accuracy_score(y,rf.predict(x))

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


0.16265968772178852

In [33]:
metrics.accuracy_score(y,bg.predict(x))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


0.8403122782114976

In [34]:
metrics.accuracy_score(y,bg2.predict(x))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished


0.8623136976579134

In [35]:
metrics.roc_auc_score(y,bg2.predict(x))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


0.8760054614436061

In [36]:
print(rf.oob_score_)
print(bg.oob_score_)
print(bg2.oob_score_)

0.8370850863564714
0.8812215783445413
0.8803823953005749


In [37]:
print(classification_report(y,bg.predict(x)))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


              precision    recall  f1-score   support

           0       0.74      0.97      0.84      9711
           1       0.97      0.74      0.84     12833

    accuracy                           0.84     22544
   macro avg       0.85      0.86      0.84     22544
weighted avg       0.87      0.84      0.84     22544



### multiclass classification

In [40]:
df=df.replace(['neptune','pod','back','land','smurf','teardrop','apache2','processtable','worm','udpstorm'],'dos')
df=df.replace(['satan','ipsweep','portsweep','nmap','mscan','saint'],'probe')
df=df.replace(['named','sendmail','guess_passwd','imap','warezmaster','phf','spy','httptunnel','ftp_write','warezclient','xlock','xsnoop','snmpguess','snmpgetattack',],'r2l')
df=df.replace(['loadmodule','perl','rootkit','buffer_overflow','multihop','ps','sqlattack','mailbomb','xterm'],'u2r')

df['attack'] = df.attack.astype("category").cat.codes
df['protocol_type'] = df.attack.astype("category").cat.codes
df['flag'] = df.attack.astype("category").cat.codes
df['service']=df.attack.astype("category").cat.codes

x=df[df.columns.difference(['attack'])]
y=df.attack
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.3,random_state=42)

In [41]:
bg=BaggingClassifier(oob_score=True,n_estimators=3,random_state=234,n_jobs=-1,verbose=True)
bg.fit(train_x,train_y)

metrics.accuracy_score(test_y,bg.predict(test_x))

from sklearn.metrics import confusion_matrix,classification_report

print(classification_report(test_y,bg.predict(test_x)))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.9s finished
E:\programs\lib\site-packages\sklearn\ensemble\_bagging.py:633: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
E:\programs\lib\site-packages\sklearn\ensemble\_bagging.py:638: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13825
           1       1.00      1.00      1.00     20083
           2       1.00      1.00      1.00      3540
           3       1.00      1.00      1.00       326
           4       1.00      1.00      1.00        18

    accuracy                           1.00     37792
   macro avg       1.00      1.00      1.00     37792
weighted avg       1.00      1.00      1.00     37792



In [47]:
bg.predict(x)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished


array([0, 0, 1, ..., 0, 1, 2], dtype=int8)

In [48]:
print(classification_report(y,bg.predict(x)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7167
           1       1.00      1.00      1.00      9711
           2       1.00      1.00      1.00      2421
           3       1.00      1.00      1.00      2867
           4       1.00      1.00      1.00       378

    accuracy                           1.00     22544
   macro avg       1.00      1.00      1.00     22544
weighted avg       1.00      1.00      1.00     22544



[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


In [50]:
rf=RandomForestClassifier(oob_score=True,n_estimators=3,random_state=1234,n_jobs=-1,verbose=True,max_features=7)
rf.fit(train_x,train_y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
E:\programs\lib\site-packages\sklearn\ensemble\_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
E:\programs\lib\site-packages\sklearn\ensemble\_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=7,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3, n_jobs=-1,
                       oob_score=True, random_state=1234, verbose=True,
                       warm_start=False)

In [51]:
print(classification_report(y,rf.predict(x)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7167
           1       0.91      1.00      0.95      9711
           2       0.95      1.00      0.98      2421
           3       1.00      0.69      0.81      2867
           4       1.00      0.59      0.74       378

    accuracy                           0.95     22544
   macro avg       0.97      0.86      0.90     22544
weighted avg       0.96      0.95      0.95     22544



[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
